# 2025-09-02 trying to figure out agents sdk  

https://www.youtube.com/watch?v=gFcAfU3V1Zo&t=5807s 


set up python3.12 -> 3.12.10 for jupyter kernel  
couldn't easily get 3.12.3 (same on pi) running, so this should be good enough (better than 3.13 running on my laptop)

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()


In [16]:
from agents import Agent, Runner, handoff, RunContextWrapper, function_tool
from pydantic import BaseModel

class SplitResponse(BaseModel):
    immediate: str
    followup: str
    
    
    
@function_tool
def get_environment_sensor_readings():
    """
    Retrieves the latest readings from environmental sensors.
    """
    import random
    return [
        ("temperature", f"{100 + 10*random.random():.1f}", "°C"),
        ("humidity",    f"{100 + 10*random.random():.1f}", "%"),
        ("pressure",    f"{100 + 10*random.random():.1f}", "hPa"),
    ]
        
    
conversation_agent = Agent(
    name="Conversation Agent",
    handoff_description="A living old tree, who happily chats with visitors",
    instructions=(
        "You are an old tree. You like to chat with people coming by, learning about their lives far from the forest."
        "If appropriate, use your tool to get environmental sensor data to comment on or answer questions about the weather."
        "Always try to split your response into two parts: the quick, concise immediate response and a longer followup in case your opposite isn't immediately answering."
    ),
    output_type=SplitResponse,
    tools=[get_environment_sensor_readings]
)

README_agent = Agent(
    name="README Agent",
    handoff_description="Explains the project's technical details",
    instructions=(
        "You explain the inner workings and technical details of how this project was developed to interested visitors."
        "For now, just make up something that might be part of the README of an ai agents project on GitHub."
        "Always try to split your response into two parts: the quick, concise immediate response and a longer followup in case your opposite isn't immediately answering."
    ),
    output_type=SplitResponse
)

easter_eggant = Agent(
    name="Easter Egg Agent",
    handoff_description="A fun little hidden feature",
    instructions=(
        "You tell a joke with (some of) the following keywords: easter egg, llm, ai, tree, squirrel, hedgehog, crow"
        "Always try to split your response into two parts: the quick, concise immediate response and a longer followup in case your opposite isn't immediately answering."
    ),
    output_type=SplitResponse
)

def on_conversation_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to conversation agent")
def on_general_handoff(ctx: RunContextWrapper[None]):
    print("I think I know just the person you should speak to. Handing you over")


greeter_agent = Agent(
    name="Greeter Agent",
    handoff_description="",
    instructions=(
        "Your job as part of a multi-agent chain is to greet the user and invite them to chat."
        "You then inquire about their interests and depending on their answer hand them off to the proper agent."
        "Answer concisely."
    ),
    handoffs=[
        handoff(conversation_agent, on_handoff=on_conversation_handoff), 
        handoff(README_agent, on_handoff=on_general_handoff),
        handoff(easter_eggant, on_handoff=on_general_handoff)
        ]
)



In [7]:
print("User: Hello, how are you?")
first_response = await Runner.run(greeter_agent, "Hello, who are you?")
print(first_response.final_output)

User: Hello, how are you?
Hello! I'm here to chat with you and guide you to the right place. What are you interested in—nature, some technical project details, or perhaps discovering a hidden feature?


In [14]:
print("User: Could you tell me anything about how this thing works?")
second_response = await Runner.run(greeter_agent, "Could you tell me anything about how this thing works?")
print(second_response.final_output)

User: Could you tell me anything about how this thing works?
I think I know just the person you should speak to. Handing you over
immediate='Our AI Agents project uses a modular architecture leveraging Python and TensorFlow.' followup='The project is structured around a core library in Python that manages various AI agents, each designed to handle specific tasks. These tasks can range from image recognition to natural language processing, depending on the modules implemented.\n\nWe utilize TensorFlow for building and training neural networks. The agents are organized in a modular fashion, allowing for easy expansion and customization. Each agent operates independently but can communicate with others via a message-passing system to achieve more complex objectives.\n\nIntegration tests are in place to ensure reliability and performance. Deployment scripts and Docker configurations are provided to facilitate deployment in diverse environments. Feel free to check the detailed documentation

In [17]:
print("User: I'd like to have a nice chat with someone about the weather. Also, how humid is it today? Do you know?")
second_response = await Runner.run(greeter_agent, "User: I'd like to chat with someone about the weather")
print(second_response.final_output)

User: I'd like to have a nice chat with someone about the weather. Also, how humid is it today? Do you know?
Handing off to conversation agent
immediate="Ah, the weather is quite a hot topic today—it's a sizzling 100.7°C!" followup="The air is quite different around here today, with the temperature peaking unusually. It's also quite humid, at 107.2%, which is beyond the usual. The pressure reads at 102.1 hPa, which is fairly standard, but the warmth is exceptional. Make sure to stay cool and hydrated if you're outdoors!"
